In [196]:
from functools import update_wrapper

def decorator(d):
    "Make function d a decorator: d wraps a function fn."
    def _d(fn):
        return update_wrapper(d(fn), fn)
    update_wrapper(_d, d)
    return _d

@decorator
def memo(f):
    """Decorator that caches the return value for each call to f(args).
    Then when called again with same args, we can just look it up."""
    cache = {}
    def _f(*args):
        try:
            return cache[args]
        except KeyError:
            cache[args] = result = f(*args)
            return result
        except TypeError:
            # some element of args can't be a dict key
            return f(args)
    _f.cache = cache
    return _f

In [232]:
# This problem deals with the one-player game foxes_and_hens. This 
# game is played with a deck of cards in which each card is labelled
# as a hen 'H', or a fox 'F'. 
# 
# A player will flip over a random card. If that card is a hen, it is
# added to the yard. If it is a fox, all of the hens currently in the
# yard are removed.
#
# Before drawing a card, the player has the choice of two actions, 
# 'gather' or 'wait'. If the player gathers, she collects all the hens
# in the yard and adds them to her score. The drawn card is discarded.
# If the player waits, she sees the next card. 


# -----------------
# User Instructions
# 
# This problem deals with the one-player game foxes_and_hens. This 
# game is played with a deck of cards in which each card is labelled
# as a hen 'H', or a fox 'F'. 
# 
# A player will flip over a random card. If that card is a hen, it is
# added to the yard. If it is a fox, all of the hens currently in the
# yard are removed.
#
# Before drawing a card, the player has the choice of two actions, 
# 'gather' or 'wait'. If the player gathers, she collects all the hens
# in the yard and adds them to her score. The drawn card is discarded.
# If the player waits, she sees the next card. 
#
# Your job is to define two functions. The first is do(action, state), 
# where action is either 'gather' or 'wait' and state is a tuple of 
# (score, yard, cards). This function should return a new state with 
# one less card and the yard and score properly updated.
#
# The second function you define, strategy(state), should return an 
# action based on the state. This strategy should average at least 
# 1.5 more points than the take5 strategy.

import random

def foxes_and_hens(strategy, foxes=7, hens=45):
    """Play the game of foxes and hens."""
    # A state is a tuple of (score-so-far, number-of-hens-in-yard, deck-of-cards)
    state = (score, yard, cards) = (0, 0, 'F'*foxes + 'H'*hens)
    while cards:
        action = strategy(state)
        state = (score, yard, cards) = do(action, state)
    return score + yard

def do(action, state):
    "Apply action to state, returning a new state."
    # Make sure you always use up one card.
    (score, yard, cards) = state
    card = random.choice(cards)
    cards_left = cards.replace(card, '', 1)
    if action == "gather":
        return (score + yard, 0, cards_left)
    elif action == "wait" and card == "F":
        return (score, 0, cards_left)
    elif action == "wait" and card == "H":
        return (score, yard + 1, cards_left)
    else:
        raise ValueError

###############################################################################
# Strategies functions
###############################################################################
def prob_fox(cards):
    "return the probabilities that next card is a fox"
    foxes = cards.count("F")
    hens = cards.count("H")
    return float(foxes) / (foxes + hens)   

def take5(state):
    "A strategy that waits until there are 5 hens in yard, then gathers."
    (score, yard, cards) = state
    if yard < 5:
        return 'wait'
    else:
        return 'gather'
    
def strategy(state, debug=False):
    "A strategy based on probabilities"
    (score, yard, cards) = state
    p_fox = prob_fox(cards)
    E_wait = p_fox * (-yard) + (1-p_fox) * (yard+1)
    E_gather = p_fox * (yard) + (1-p_fox) * (yard-1)
    return "wait" if E_wait > E_gather else "gather" 
    
def Q_foxes(state, action, utilityFun):  
    "Quality: The expected value of choosing action in state."
    (score, yard, cards) = state
    foxes = cards.count("F")
    hens = cards.count("H")
    p_fox = prob_fox(cards)
    
    Q = 0
    if action == "gather":
        if foxes:
            Q += p_fox * utilityFun( (score+yard, 0, cards.replace('F', '', 1)) )
        if hens:
            Q += (1-p_fox) * utilityFun( (score+yard, 0, cards.replace('H', '', 1)) )
    elif action == "wait":
        if foxes:
            Q += p_fox * utilityFun( (score, 0, cards.replace('F', '', 1)) )
        if hens:
            Q += (1-p_fox) * utilityFun( (score, yard+1, cards.replace('H', '', 1)) )
    return Q

@memo
def Util(state):
    "The utility of a state"
    (score, yard, cards) = state
    # print state
    if not cards:
        return (score + yard)
    else:
        return max(Q_foxes(state, action, Util) for action in ["wait", "gather"])

def optim_strategy(state):
    return max(["wait", "gather"], key=lambda action: Q_foxes(state, action, Util))

############################################################################### 
foxes_and_hens(optim_strategy)

32

In [234]:
def average_score(strategy, N=1000):
    return sum(foxes_and_hens(strategy) for _ in range(N)) / float(N)

def superior(A, B=take5):
    "Does strategy A have a higher average score than B, by more than 1.5 point?"
    avg_diff =  average_score(A) - average_score(B)
    print "average difference:", avg_diff
    return avg_diff > 1.5

def test():
    gather = do('gather', (4, 5, 'F'*4 + 'H'*10))
    assert (gather == (9, 0, 'F'*3 + 'H'*10) or 
            gather == (9, 0, 'F'*4 + 'H'*9))
    
    wait = do('wait', (10, 3, 'FFHH'))
    assert (wait == (10, 4, 'FFH') or
            wait == (10, 0, 'FHH'))
    
    assert superior(optim_strategy)
    return 'tests pass'

import time
start_time = time.time()
print test()
print "cache length:", len(Util.cache)
print "time elapsed:", time.time() - start_time

average difference: 3.2877
tests pass
cache length: 137332
time elapsed: 7.27539801598
